## Collatz components notebook

In [8]:
"""
This notebook analyses core components of collatz 
sequences and their relationship: 
a.) k**i 
b.) beta_i = 1 + 1/(k*xi)
c.) alpha_i

The alphas of the sequence are compared with a predicted alphas values.
"""

# Fix possible import problems
import sys
sys.path.append("..")

# Imports
import random as rnd
from math import log2
import pandas as pd
from collatz import generator as gen
from collatz import commons as com

# Configuration
MAX_VALUE = 101
K_FACTOR = 3
MAX_ITERATIONS = 100
LOG_MODE = None
PRINT_TABLE = True

START_VALUE = rnd.randint(1, MAX_VALUE)

if START_VALUE % 2 == 0:
    START_VALUE = START_VALUE + 1

# START_VALUE = 13

pd.set_option('display.expand_frame_repr', False)
pd.set_option('display.max_rows', 10000)
pd.set_option('display.expand_frame_repr', False)

# Generate Collatz sequence
analysis_frame = gen.generate_odd_collatz_sequence(
    start_value=START_VALUE, k=K_FACTOR, max_iterations=MAX_ITERATIONS)

analysis_frame = analysis_frame[:-1]

# Derive new fields
analysis_frame["v_1"] = START_VALUE
analysis_frame["n"] = analysis_frame.index + 1
analysis_frame["kn_log"] = log2(K_FACTOR) * analysis_frame["n"]
analysis_frame["beta_i"] = 1 + 1 / (K_FACTOR * analysis_frame["collatz"]) 
analysis_frame["beta"] = analysis_frame["beta_i"].cumprod()

analysis_frame["alpha_i"] = analysis_frame["next_collatz"].apply(com.trailing_zeros)
analysis_frame["alpha_i"] = analysis_frame["alpha_i"].astype("int64")
analysis_frame["alpha"] = analysis_frame["alpha_i"].cumsum()
analysis_frame["alpha_cycle"] = (log2(K_FACTOR) * analysis_frame["n"]).astype('int64') + 1
analysis_frame["alpha_max"] = \
    log2(START_VALUE) + (analysis_frame["n"] * log2(K_FACTOR))
analysis_frame["alpha_max"] = analysis_frame["alpha_max"].astype('int64') + 1

analysis_frame["v_i_hyp"] = (analysis_frame["v_1"] * K_FACTOR**analysis_frame["n"])
analysis_frame["v_i_hyp"] = analysis_frame["v_i_hyp"] * analysis_frame["beta"]
analysis_frame["v_i_hyp"] = analysis_frame["v_i_hyp"] / 2**analysis_frame["alpha_max"]

analysis_frame["beta_hyp"] = analysis_frame["alpha_cycle"] - analysis_frame["kn_log"]

analysis_frame["v_i_bin"] = analysis_frame["collatz"].apply(com.to_binary)
analysis_frame["v_i_1_bin"] = analysis_frame["next_odd"].apply(com.to_binary)

analysis_frame["gamma_i"] = 1 / (K_FACTOR * analysis_frame["collatz"])
analysis_frame["gamma"] = analysis_frame["gamma_i"].cumsum()

# Possible set log mode
if LOG_MODE:
    analysis_frame["v_1"] = analysis_frame["v_1"].apply(LOG_MODE)
    analysis_frame["collatz"] = analysis_frame["collatz"].apply(LOG_MODE)
    analysis_frame["next_odd"] = analysis_frame["next_odd"].apply(LOG_MODE)
    analysis_frame["beta_i"]= analysis_frame["beta_i"].apply(LOG_MODE)
    analysis_frame["beta"]= analysis_frame["beta"].apply(LOG_MODE)
    analysis_frame["beta"]= analysis_frame["gamma_i"].apply(LOG_MODE)
    analysis_frame["beta"]= analysis_frame["gamma"].apply(LOG_MODE)
else:
    analysis_frame["beta_hyp"] = 2**analysis_frame["beta_hyp"]

# Validate alpha max & alpha pred
final_alpha = analysis_frame["alpha"].max()
final_alpha_max = analysis_frame["alpha_max"].max()

alpha_max_valid = final_alpha == final_alpha_max

# Get max beta
beta_max = analysis_frame["beta"].max()

# Print results
print_frame = analysis_frame[[
    "n", "v_1", "collatz", "next_odd",
    "alpha_i", "alpha", "alpha_cycle", "alpha_max", 
    "beta_i", "beta", "gamma_i", "gamma"]]

print_frame.columns = [
    "n","v_1", "v_i", "v_i+",
    "a_i", "a", "a_cycle", "a_max", 
    "b_i", "b", "g_i", "g"]

print("Start value:", START_VALUE, 
      " K:", K_FACTOR, 
      " Beta max: ", round(beta_max, 4), " ",
       " Alphas valid:", alpha_max_valid, "\n")

if PRINT_TABLE:
    print(print_frame.to_string(index=False), "\n")


Start value: 67  K: 3  Beta max:  1.1927    Alphas valid: True 

 n  v_1  v_i  v_i+  a_i   a  a_cycle  a_max       b_i         b       g_i         g
 1   67   67   101    1   1        2      8  1.004975  1.004975  0.004975  0.004975
 2   67  101    19    4   5        4     10  1.003300  1.008292  0.003300  0.008275
 3   67   19    29    1   6        5     11  1.017544  1.025981  0.017544  0.025819
 4   67   29    11    3   9        7     13  1.011494  1.037774  0.011494  0.037314
 5   67   11    17    1  10        8     14  1.030303  1.069222  0.030303  0.067617
 6   67   17    13    2  12       10     16  1.019608  1.090187  0.019608  0.087224
 7   67   13     5    3  15       12     18  1.025641  1.118140  0.025641  0.112865
 8   67    5     1    4  19       13     19  1.066667  1.192683  0.066667  0.179532 

